## Project: Maritime Fuels LCA
### Notebook 3: Doing the contribution analyses
#### This notebook is developed by Megan Roux and Massimo Pizzol, and includes adaptions of code provided by Massimo Pizzol, Nils Thonemann and Chris Mutel

This notebook relates to work done for the publication 'Consequential LCA of alternative maritime fuels' which can be found at: doi

Begin by importing all relevant packages (ensure you are in the right environment):

In [1]:
# Note: must be environment bw2
import brightway2 as bw
import pandas as pd
import numpy as np
import matplotlib
from lci_to_bw2 import * # this file has to be in the same folder as the notebook and raw data

Set the project (same as in Notebook 1):

In [2]:
bw.projects.set_current("fuels") # setting project
# bw.projects.copy_project("fuels") # making a copy

Now check which databases are in the project:
(Should be biopshere3, ecoinvent 3.9 conseq, Fuels_WtT, Fuels_TtW and Fuels_WtW)

In [3]:
bw.databases

Databases dictionary with 23 objects, including:
	Fuels_db_SA1
	Fuels_db_SA1a
	Fuels_db_SA1b
	Fuels_db_SA1c
	Fuels_db_SA1d
	Fuels_db_SA1e
	Fuels_db_SA1f
	Fuels_db_SA2
	Fuels_db_SA2a
	Fuels_db_SA2b
Use `list(this object)` to get the complete list.

In [4]:
ei39db = bw.Database("ecoinvent 3.9 conseq")
biodb = bw.Database("biosphere3")

In [5]:
Fuels_WtT = [(act['database'], act['code']) for act in bw.Database('Fuels_db_WtT_MJ')]
Fuels_TtW = [(act['database'], act['code']) for act in bw.Database('Fuels_db_TtW_MJ')]
Fuels_WtW = [(act['database'], act['code']) for act in bw.Database('Fuels_db_WtW_MJ')]

Check the activities in your foreground database:

In [12]:
Fuels_WtT
# Here, all activities are included, even capital goods and eh2. Remove these from the figs
# Pick any of the foreground databases

[('Fuels_db_WtT_MJ', 'DME'),
 ('Fuels_db_WtT_MJ', 'VLSFO'),
 ('Fuels_db_WtT_MJ', 'Straw'),
 ('Fuels_db_WtT_MJ', 'NH3_CCS'),
 ('Fuels_db_WtT_MJ', 'eMeOH_bio'),
 ('Fuels_db_WtT_MJ', 'bioMeOH'),
 ('Fuels_db_WtT_MJ', 'eNH3'),
 ('Fuels_db_WtT_MJ', 'DAC'),
 ('Fuels_db_WtT_MJ', 'Wood'),
 ('Fuels_db_WtT_MJ', 'PO_slow'),
 ('Fuels_db_WtT_MJ', 'NH3_FGC'),
 ('Fuels_db_WtT_MJ', 'LNG'),
 ('Fuels_db_WtT_MJ', 'eH2'),
 ('Fuels_db_WtT_MJ', 'eMeOH_DAC'),
 ('Fuels_db_WtT_MJ', 'PO_fast'),
 ('Fuels_db_WtT_MJ', 'eMeOH_DOC')]

We want to test that everything has worked, so we will do a quick LCA over one of the activities for one impact category.

First, choose an LCIA method (e.g. EF3.1 in the case of this study):

In [10]:
EF31 = [method for method in bw.methods if "EF v3.1" in str(method) 
        and "no LT" not in str(method)
        and "EN15804" not in str(method)]
# EF31

A test over one activity and one impact category to see if the LCA works:

In [6]:
# For eMeOH_DAC climate change impacts using EF31
mymethod = EF31[1]
print(mymethod)
Fuels_WtT[-6]
myact = Fuels_WtT[-6]
print(myact)
functional_unit = {myact: 1} 
lca = bw.LCA(functional_unit, mymethod)
lca.lci()
lca.lcia()
print(lca.score)

('EF v3.1', 'climate change', 'global warming potential (GWP100)')
('Fuels_db_WtT_MJ', 'NH3_CCS')
0.2039004753589117


The function to perform the LCA over all activities in all databases:

In [7]:
def dolcacalc(myact, mydemand, mymethod):
    my_fu = {myact: mydemand} 
    lca = bw.LCA(my_fu, mymethod)
    lca.lci()
    lca.lcia()
    return lca.score

# For WtT
def getLCAresults(list_acts, mymethod):
    
    all_activities = []
    results = []
    for a in list_acts:
        act = bw.Database(a[0]).get(a[1])
        all_activities.append(act['name'])
        results.append(dolcacalc(act,1,mymethod)) # 1 stays for one unit of each process
        #print(act['name'])
     
    results_dict = dict(zip(all_activities, results))
    
    return results_dict

Running the LCA calculations:

In [62]:
Results_WtT_EF31 = []
for m in EF31:
    results_all_fuels_WtT = getLCAresults(Fuels_WtT,m) # total impact per tech
    Results_WtT_EF31.append(results_all_fuels_WtT)

Results_TtW_EF31 = []
for m in EF31:
    results_all_fuels_TtW = getLCAresults(Fuels_TtW,m) # total impact per tech
    Results_TtW_EF31.append(results_all_fuels_TtW)

Making everything nice and pretty and exporting to Excel:

In [17]:
methods_names = []
for m in EF31:
    m_name = ' '.join(m)
    methods_names.append(m_name)

### Contribution Analysis

Making a dictionary of all exchanges in the WtT database:

In [8]:
mymethod = EF31[1]

ca_dict = {}

for act in bw.Database('Fuels_db_WtT_MJ'):
    
    exc_list = []
    contr_list = []

    for exc in list(act.exchanges()):
        
        if exc['type'] == 'biosphere':
            
            col = lca.activity_dict[exc['output']] # find column index of A matrix for the activity
            row = lca.biosphere_dict[exc['input']] # find row index of B matrix for the exchange
            contr_score = lca.biosphere_matrix[row,col] * lca.characterization_matrix[row,row]
            contr_list.append((exc['input'], exc['type'], exc['amount'], contr_score))
            
        elif exc['type'] == 'substitution':
            
            contr_score = dolcacalc(bw.Database(exc['input'][0]).get(exc['input'][1]), exc['amount'], mymethod)
            contr_list.append((exc['input'], exc['type'], exc['amount'], -contr_score))
            
        else:
            
            contr_score = dolcacalc(bw.Database(exc['input'][0]).get(exc['input'][1]), exc['amount'], mymethod)
            contr_list.append((exc['input'], exc['type'], exc['amount'], contr_score))
        
    ca_dict[act['code']] =  contr_list

In [19]:
ca_dict['eMeOH_bio']

[(('Fuels_db_WtT_MJ', 'eH2'), 'technosphere', 0.00928, 0.019352710626128998),
 (('ecoinvent 3.9 conseq', '206c622c567b1abfcfea904962336d32'),
  'technosphere',
  0.0404,
  0.00925015804487905),
 (('ecoinvent 3.9 conseq', '6af5d75e12ba4829741c76bb3992302f'),
  'technosphere',
  1.76e-12,
  8.669562210592507e-05),
 (('ecoinvent 3.9 conseq', '7f73767288699c6c41571ef836d4e0d7'),
  'technosphere',
  0.00475,
  5.609869319744448e-05),
 (('Fuels_db_WtT_MJ', 'bioMeOH'), 'technosphere', 0.516, 0.049667541489377284),
 (('biosphere3', '73ed05cc-9727-4abf-9516-4b5c0fe54a16'),
  'biosphere',
  -0.692,
  -0.0),
 (('Fuels_db_WtT_MJ', 'eMeOH_bio'), 'production', 1.0, 0.07841320125244684)]

In [20]:
df = pd.DataFrame(ca_dict['eMeOH_bio'], columns = ['input','type','amount','contribution'])
df

,input,type,amount,contribution
0,"(Fuels_db_WtT_MJ, eH2)",technosphere,9.280000e-03,0.019353
1,"(ecoinvent 3.9 conseq, 206c622c567b1abfcfea904...",technosphere,4.040000e-02,0.009250
2,"(ecoinvent 3.9 conseq, 6af5d75e12ba4829741c76b...",technosphere,1.760000e-12,0.000087
3,"(ecoinvent 3.9 conseq, 7f73767288699c6c41571ef...",technosphere,4.750000e-03,0.000056
4,"(Fuels_db_WtT_MJ, bioMeOH)",technosphere,5.160000e-01,0.049668
5,"(biosphere3, 73ed05cc-9727-4abf-9516-4b5c0fe54...",biosphere,-6.920000e-01,-0.000000
6,"(Fuels_db_WtT_MJ, eMeOH_bio)",production,1.000000e+00,0.078413


In [21]:
print(df.loc[df['type'] == 'production']['contribution'].sum())
print(df.loc[df['type'] != 'production']['contribution'].sum())

0.07841320125244684
0.0784132044756887


In [22]:
df['%_contribution'] = 100 * df['contribution'] / df.loc[df['type'] == 'production']['contribution'].sum()
df

,input,type,amount,contribution,%_contribution
0,"(Fuels_db_WtT_MJ, eH2)",technosphere,9.280000e-03,0.019353,24.680424
1,"(ecoinvent 3.9 conseq, 206c622c567b1abfcfea904...",technosphere,4.040000e-02,0.009250,11.796685
2,"(ecoinvent 3.9 conseq, 6af5d75e12ba4829741c76b...",technosphere,1.760000e-12,0.000087,0.110563
3,"(ecoinvent 3.9 conseq, 7f73767288699c6c41571ef...",technosphere,4.750000e-03,0.000056,0.071542
4,"(Fuels_db_WtT_MJ, bioMeOH)",technosphere,5.160000e-01,0.049668,63.340790
5,"(biosphere3, 73ed05cc-9727-4abf-9516-4b5c0fe54...",biosphere,-6.920000e-01,-0.000000,-0.000000
6,"(Fuels_db_WtT_MJ, eMeOH_bio)",production,1.000000e+00,0.078413,100.000000


In [23]:
df.to_excel('Results_CA_eMeOH_bio.xlsx')

### Monte Carlo simulations

In [ ]:
# Uncertainty type: an interger that tells you what type of uncertainty (see website for list). 2 = lognormal distribution https://docs.brightway.dev/en/latest/content/theory/uncertainty.html
# loc: LN(geometric mean)
# scale: LN(geometric standard deviation)
# negative: is the value negative (TRUE) or not (FALSE)

# https://github.com/PoutineAndRosti/Brightway-Seminar-2017/blob/master/Day%201%20PM/Brightway%20and%20uncertainty.ipynb
# https://stats-arrays.readthedocs.io/en/latest/index.html

# What I did: 
# Calculated the natural log of the median and natural log of the geometric standard deviation and used these for 'loc' and 'scale' values

In [11]:
mymethod = EF31[1]

Selecting a number of fuels to do the MC over

In [16]:
Fuels_WtT_2 = ['eMeOH_DAC',
                'eMeOH_bio',
                'bioMeOH',
                'NH3_CCS',
                'NH3_FGC',
                'eNH3',
                'DME',
                'PO_fast',
                'PO_slow',
                'LNG' , 
                'VLSFO']

Fuels_TtW_2 = ['MeOH_combustion',
                'NH3_combustion',
                'DME_combustion',
                'LNG_combustion',
                'PO_combustion',
                'VLSFO_combustion']

Fuels_WtW_2 = ['eMeOH_DAC_WtW_MJ',
                'eMeOH_bio_WtW_MJ',
                'bioMeOH_WtW_MJ',
                'NH3_CCS_WtW_MJ',
                'NH3_FGC_WtW_MJ',
                'eNH3_WtW_MJ',
                'DME_WtW_MJ',
                'PO_fast_WtW_MJ',
                'PO_slow_WtW_MJ',
                'LNG_WtW_MJ' , 
                'VLSFO_WtW_MJ']

In [17]:
fus = [] # list of functional units
for a in Fuels_WtT_2:
    act = bw.Database('Fuels_db_WtT_MJ').get(a)
    functional_unit = {act: 1} # one unit of each process
    fus.append(functional_unit)

In [89]:
mc = bw.MonteCarloLCA(fus[0], mymethod) # important to initialize the MC simulation
next(mc)

0.16959096081602226

In [90]:
mc.redo_lcia(fus[0]) #just a test
print(mc.score)
mc.redo_lcia(fus[1])
print(mc.score)

0.16959096081602226
0.09694506217680424


In [91]:
# Now the real simulation (takes time)
iterations = 100
simulations = []

for _ in range(iterations):
    print(_)
    next(mc)
    mcresults = []    
    for i in fus:
        mc.redo_lcia(i)
        mcresults.append(mc.score)
    simulations.append(mcresults)
    

simulations

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


[[0.16868904250466332,
  0.0808993189489831,
  0.11750620851492338,
  0.3557538776811862,
  0.054841150565151854,
  0.17555967401291092,
  -0.04311936281905728,
  -0.1047701989352463,
  0.03715611946899392,
  0.03270539997390437],
 [0.21832720628103844,
  0.12821148000099225,
  0.19339784985473613,
  0.36615084826770605,
  0.09725189415651714,
  0.2718302181670803,
  0.022423924607128755,
  0.08140824625492536,
  0.013527919750172198,
  0.04471964177404559],
 [0.15156016422089233,
  0.1051311960095242,
  0.1437888300787665,
  0.4258259816050211,
  0.06085212894813196,
  0.21379388872453564,
  0.02201004856256538,
  0.11857886947585977,
  0.027892968118342527,
  0.03238605171675122],
 [0.16373494870198338,
  0.12546494860264185,
  0.18142530852829986,
  0.39621766458431174,
  0.06162030076028354,
  0.24204711979329577,
  0.03345781685537022,
  0.9864418296048975,
  0.015481677331278603,
  0.03811285654837396],
 [0.15651460261624822,
  0.11627735048073735,
  0.17103526108958367,
  0.6365

In [92]:
df = pd.DataFrame(simulations, columns = Fuels_WtT_2)
df.head()

,eMeOH_DAC,eMeOH_bio,bioMeOH,NH3_CCS,eNH3,DME,PO_fast,PO_slow,LNG,VLSFO
0,0.168689,0.080899,0.117506,0.355754,0.054841,0.175560,-0.043119,-0.104770,0.037156,0.032705
1,0.218327,0.128211,0.193398,0.366151,0.097252,0.271830,0.022424,0.081408,0.013528,0.044720
2,0.151560,0.105131,0.143789,0.425826,0.060852,0.213794,0.022010,0.118579,0.027893,0.032386
3,0.163735,0.125465,0.181425,0.396218,0.061620,0.242047,0.033458,0.986442,0.015482,0.038113
4,0.156515,0.116277,0.171035,0.636581,0.069483,0.246644,0.087921,0.170671,0.051754,0.058704


In [93]:
df.to_excel('Results_Fuels_WtT_MJ_EF31_MC.xlsx')

Now for the Tank-to-Wake phase:

In [20]:
Fuels_TtW_MC = [] # list of functional units
for a in Fuels_TtW_2:
    act = bw.Database('Fuels_db_TtW_MJ').get(a)
    functional_unit = {act: 1} # one unit of each process
    Fuels_TtW_MC.append(functional_unit)

In [21]:
mc = bw.MonteCarloLCA(Fuels_TtW_MC[0], mymethod) # important to initialize the MC simulation
next(mc)

0.006817777397082846

In [22]:
mc.redo_lcia(Fuels_TtW_MC[0]) #just a test
print(mc.score)
mc.redo_lcia(Fuels_TtW_MC[1])
print(mc.score)

0.006817777397082846
0.013268168543377415


In [23]:
# Now the real simulation (takes time)
iterations = 100
simulations = []

for _ in range(iterations):
    print(_)
    next(mc)
    mcresults = []    
    for i in Fuels_TtW_MC:
        mc.redo_lcia(i)
        mcresults.append(mc.score)
    simulations.append(mcresults)
    

simulations

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


[[0.006875714699829115,
  0.013323914286085054,
  0.008231647099887707,
  0.12895243317895536,
  0.007544624149530114,
  0.18710200920488448],
 [0.007419918687606605,
  0.014657391508940803,
  0.008410817846526367,
  0.09744715532349477,
  0.008393150902125515,
  0.18633841920368094],
 [0.006582539649810318,
  0.012948275869226918,
  0.007708282386679486,
  0.1415333577162857,
  0.007819420366707706,
  0.153713240418286],
 [0.0077590289317427345,
  0.015883777140478367,
  0.008965888365033096,
  0.1400144329264187,
  0.007951249110223463,
  0.16067113959572574],
 [0.006051451265199816,
  0.006592832538846346,
  0.006974434640391786,
  0.15915244933533648,
  0.006341478816787696,
  0.19472769744827317],
 [0.0066903621059837695,
  0.007993271890723188,
  0.007535730040336953,
  0.21501043698738875,
  0.0074435958238887176,
  0.1788742411797479],
 [0.006505297830499894,
  0.008079039241256124,
  0.00788952658162883,
  0.13836605379546743,
  0.006896519804724161,
  0.15402260994795985],
 [

In [24]:
df = pd.DataFrame(simulations, columns = Fuels_TtW_2)
df.head()

,MeOH_combustion,NH3_combustion,DME_combustion,LNG_combustion,PO_combustion,VLSFO_combustion
0,0.006876,0.013324,0.008232,0.128952,0.007545,0.187102
1,0.007420,0.014657,0.008411,0.097447,0.008393,0.186338
2,0.006583,0.012948,0.007708,0.141533,0.007819,0.153713
3,0.007759,0.015884,0.008966,0.140014,0.007951,0.160671
4,0.006051,0.006593,0.006974,0.159152,0.006341,0.194728


In [25]:
df.to_excel('Results_Fuels_TtW_MJ_EF31_MC.xlsx')

For the 'well-to-wake' scenario:

In [18]:
Fuels_WtW_MC = [] # list of functional units
for a in Fuels_WtW_2:
    act = bw.Database('Fuels_db_WtW_MJ').get(a)
    functional_unit = {act: 1} # one unit of each process
    Fuels_WtW_MC.append(functional_unit)

In [19]:
mc = bw.MonteCarloLCA(Fuels_WtW_MC[0], mymethod) # important to initialize the MC simulation
next(mc)

0.18599531201571126

In [20]:
mc.redo_lcia(Fuels_WtW_MC[0]) #just a test
print(mc.score)
mc.redo_lcia(Fuels_WtW_MC[1])
print(mc.score)

0.18599531201571126
0.09543995609328679


In [21]:
# Now the real simulation (takes time)
iterations = 100
simulations = []

for _ in range(iterations):
    print(_)
    next(mc)
    mcresults = []    
    for i in Fuels_WtW_MC:
        mc.redo_lcia(i)
        mcresults.append(mc.score)
    simulations.append(mcresults)
    

simulations

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


[[0.15791934865479276,
  0.1055549400414092,
  0.12284914343561526,
  0.2467378644434115,
  0.3795091481745687,
  0.10963136442667498,
  0.1757997175145866,
  0.041679852556453956,
  -0.1707494724068262,
  0.7026179568412877,
  0.22033973330131829],
 [0.20426663410523632,
  0.12237494340097232,
  0.1459072617648649,
  0.28733171653289996,
  0.36759693165401847,
  0.1527345892129914,
  0.20335817722511745,
  0.04973144362737968,
  0.1799924685638785,
  0.20709805298884737,
  0.21473908898941074],
 [0.18452789002270414,
  0.11428783691880336,
  0.1516641049319423,
  0.29037555166056267,
  0.3468999844734977,
  0.12578014933148898,
  0.20621963086782794,
  0.08616953539144348,
  0.01909951586652072,
  0.13634951566940112,
  0.23793197923745435],
 [0.22943350224564252,
  0.15967377194129798,
  0.16025884491126852,
  0.35347092450566286,
  0.32561925274704356,
  0.21477816397114113,
  0.21306162556186398,
  0.06480275360855056,
  0.04537750365077704,
  0.14742468192071265,
  0.1920356683602

In [22]:
df = pd.DataFrame(simulations, columns = Fuels_WtW_2)
df.head()

,eMeOH_DAC_WtW_MJ,eMeOH_bio_WtW_MJ,bioMeOH_WtW_MJ,NH3_CCS_WtW_MJ,NH3_FGC_WtW_MJ,eNH3_WtW_MJ,DME_WtW_MJ,PO_fast_WtW_MJ,PO_slow_WtW_MJ,LNG_WtW_MJ,VLSFO_WtW_MJ
0,0.157919,0.105555,0.122849,0.246738,0.379509,0.109631,0.175800,0.041680,-0.170749,0.702618,0.220340
1,0.204267,0.122375,0.145907,0.287332,0.367597,0.152735,0.203358,0.049731,0.179992,0.207098,0.214739
2,0.184528,0.114288,0.151664,0.290376,0.346900,0.125780,0.206220,0.086170,0.019100,0.136350,0.237932
3,0.229434,0.159674,0.160259,0.353471,0.325619,0.214778,0.213062,0.064803,0.045378,0.147425,0.192036
4,0.173049,0.100173,0.121168,0.295841,0.342623,0.116911,0.162214,0.079821,1.082476,0.287580,0.218649


In [23]:
df.to_excel('Results_Fuels_WtW_MJ_EF31_MC.xlsx')

## Sensitivity Analysis

We will conduct three SAs: 
SA1: Changing electricity (grid and wind) to A) NL, B) DE, C) BE and D) ES, and E) Fully renewable
SA2: Changing the counterfactual of wood and sraw residues to A) use for bio-electricity and B) use for heat and electricity
SA3: Changing biochar use for PO to fertilizer
SA4: Changing the emissions from NH3 combustion
SA5: 

In [10]:
SA1a = pd.read_csv('Fuels_db_SA1a.csv', header = 0, sep = ";")
SA1b = pd.read_csv('Fuels_db_SA1b.csv', header = 0, sep = ";")
SA1c = pd.read_csv('Fuels_db_SA1c.csv', header = 0, sep = ";")
SA1d = pd.read_csv('Fuels_db_SA1d.csv', header = 0, sep = ";")
SA1e = pd.read_csv('Fuels_db_SA1e.csv', header = 0, sep = ";")
SA1f = pd.read_csv('Fuels_db_SA1f.csv', header = 0, sep = ";")

In [11]:
SA1a = SA1a.drop('Notes', 1)  # remove the columns not needed
SA1a['Exchange uncertainty type'] = SA1['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
SA1b = SA1b.drop('Notes', 1)  # remove the columns not needed
SA1b['Exchange uncertainty type'] = SA1['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
SA1c = SA1c.drop('Notes', 1)  # remove the columns not needed
SA1c['Exchange uncertainty type'] = SA1['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
SA1d = SA1d.drop('Notes', 1)  # remove the columns not needed
SA1d['Exchange uncertainty type'] = SA1['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
SA1e = SA1e.drop('Notes', 1)  # remove the columns not needed
SA1e['Exchange uncertainty type'] = SA1['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
SA1f = SA1f.drop('Notes', 1)  # remove the columns not needed
SA1f['Exchange uncertainty type'] = SA1['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers

C:\Users\merou\AppData\Local\Temp\ipykernel_1936\719344963.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  SA1a = SA1a.drop('Notes', 1)  # remove the columns not needed
C:\Users\merou\AppData\Local\Temp\ipykernel_1936\719344963.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  SA1b = SA1b.drop('Notes', 1)  # remove the columns not needed
C:\Users\merou\AppData\Local\Temp\ipykernel_1936\719344963.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  SA1c = SA1c.drop('Notes', 1)  # remove the columns not needed
C:\Users\merou\AppData\Local\Temp\ipykernel_1936\719344963.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  SA1d = SA1d.drop('

In [12]:
Fuels_SA1a = lci_to_bw2(SA1a) # a function from the lci_to_bw2 module
Fuels_SA1b = lci_to_bw2(SA1b) # a function from the lci_to_bw2 module
Fuels_SA1c = lci_to_bw2(SA1c) # a function from the lci_to_bw2 module
Fuels_SA1d = lci_to_bw2(SA1d) # a function from the lci_to_bw2 module
Fuels_SA1e = lci_to_bw2(SA1e) # a function from the lci_to_bw2 module
Fuels_SA1f = lci_to_bw2(SA1f) # a function from the lci_to_bw2 module

In [13]:
if 'Fuels_db_SA1a' in bw.databases: del bw.databases['Fuels_db_SA1a']
Fuels_db_SA1a = bw.Database('Fuels_db_SA1a') # it works because the database name in the excel file is the same
Fuels_db_SA1a.write(Fuels_SA1a)

if 'Fuels_db_SA1b' in bw.databases: del bw.databases['Fuels_db_SA1b']
Fuels_db_SA1b = bw.Database('Fuels_db_SA1b') # it works because the database name in the excel file is the same
Fuels_db_SA1b.write(Fuels_SA1b)

if 'Fuels_db_SA1c' in bw.databases: del bw.databases['Fuels_db_SA1c']
Fuels_db_SA1c = bw.Database('Fuels_db_SA1c') # it works because the database name in the excel file is the same
Fuels_db_SA1c.write(Fuels_SA1c)

if 'Fuels_db_SA1d' in bw.databases: del bw.databases['Fuels_db_SA1d']
Fuels_db_SA1d = bw.Database('Fuels_db_SA1d') # it works because the database name in the excel file is the same
Fuels_db_SA1d.write(Fuels_SA1d)

if 'Fuels_db_SA1e' in bw.databases: del bw.databases['Fuels_db_SA1e']
Fuels_db_SA1e = bw.Database('Fuels_db_SA1e') # it works because the database name in the excel file is the same
Fuels_db_SA1e.write(Fuels_SA1e)

if 'Fuels_db_SA1f' in bw.databases: del bw.databases['Fuels_db_SA1f']
Fuels_db_SA1f = bw.Database('Fuels_db_SA1f') # it works because the database name in the excel file is the same
Fuels_db_SA1f.write(Fuels_SA1f)

Writing activities to SQLite3 database:
0% [###############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/31/2024 14:41:30
  Finished: 07/31/2024 14:41:30
  Total time elapsed: 00:00:00
  CPU %: 50.40
  Memory %: 2.46


Writing activities to SQLite3 database:
0% [###############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/31/2024 14:41:31
  Finished: 07/31/2024 14:41:31
  Total time elapsed: 00:00:00
  CPU %: 50.40
  Memory %: 2.46


Writing activities to SQLite3 database:
0% [###############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
Writing activities to SQLite3 database:


...

Title: Writing activities to SQLite3 database:
  Started: 07/31/2024 14:41:31
  Finished: 07/31/2024 14:41:31
  Total time elapsed: 00:00:00
  CPU %: 50.40
  Memory %: 2.47


Writing activities to SQLite3 database:
0% [###############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/31/2024 14:41:31
  Finished: 07/31/2024 14:41:31
  Total time elapsed: 00:00:00
  CPU %: 100.80
  Memory %: 2.47


Writing activities to SQLite3 database:
0% [###############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/31/2024 14:41:32
  Finished: 07/31/2024 14:41:32
  Total time elapsed: 00:00:00
  CPU %: 66.50
  Memory %: 2.47


In [21]:
Fuels_SA1a = [(act['database'], act['code']) for act in bw.Database('Fuels_db_SA1a')]
Fuels_SA1b = [(act['database'], act['code']) for act in bw.Database('Fuels_db_SA1b')]
Fuels_SA1c = [(act['database'], act['code']) for act in bw.Database('Fuels_db_SA1c')]
Fuels_SA1d = [(act['database'], act['code']) for act in bw.Database('Fuels_db_SA1d')]
Fuels_SA1e = [(act['database'], act['code']) for act in bw.Database('Fuels_db_SA1e')]
Fuels_SA1f = [(act['database'], act['code']) for act in bw.Database('Fuels_db_SA1f')]
Fuels_SA1f

[('Fuels_db_SA1f', 'DME'),
 ('Fuels_db_SA1f', 'PO_slow'),
 ('Fuels_db_SA1f', 'eH2'),
 ('Fuels_db_SA1f', 'Wood'),
 ('Fuels_db_SA1f', 'NH3_CCS'),
 ('Fuels_db_SA1f', 'eMeOH_DAC'),
 ('Fuels_db_SA1f', 'PO_fast'),
 ('Fuels_db_SA1f', 'DAC'),
 ('Fuels_db_SA1f', 'eNH3'),
 ('Fuels_db_SA1f', 'eMeOH_bio'),
 ('Fuels_db_SA1f', 'VLSFO'),
 ('Fuels_db_SA1f', 'Straw'),
 ('Fuels_db_SA1f', 'bioMeOH'),
 ('Fuels_db_SA1f', 'LNG'),
 ('Fuels_db_SA1f', 'eMeOH_DOC')]

In [23]:
mymethod = EF31[1]
print(mymethod)
Fuels_SA1f[-7]
myact = Fuels_SA1f[-7]
print(myact)
functional_unit = {myact: 1} 
lca = bw.LCA(functional_unit, mymethod)
lca.lci()
lca.lcia()
print(lca.score)

('EF v3.1', 'climate change', 'global warming potential (GWP100)')
('Fuels_db_SA1f', 'eNH3')
0.08244623682123654


In [152]:
Results_SA1a = []
for m in EF31:
    results_SA1a = getLCAresults(Fuels_SA1a,m) # total impact per tech
    Results_SA1a.append(results_SA1a)

In [153]:
methods_names = []
for m in EF31:
    m_name = ' '.join(m)
    methods_names.append(m_name)

In [154]:
my_output = pd.DataFrame(Results_SA1, index=methods_names)
my_output.head()

,Straw,NH3_CCS,LNG,eMeOH_bio,eH2,VLSFO,eMeOH_DOC,DME,bioMeOH,eMeOH_DAC,Wood,eNH3,PO_slow,DAC,PO_fast
EF v3.1 acidification accumulated exceedance (AE),0.000315,0.000312,0.000030,0.000458,0.010953,0.000131,-4853.079828,0.000851,0.000665,0.000498,0.000686,0.000260,0.001070,0.000006,0.001004
EF v3.1 climate change global warming potential (GWP100),0.141122,0.136245,0.021443,0.078901,2.085421,0.039559,16824.661223,0.153695,0.111870,0.132887,0.298683,0.047883,0.002013,0.003847,0.039376
EF v3.1 climate change: biogenic global warming potential (GWP100),0.000015,-0.007742,0.000001,0.000061,0.001711,0.000008,20.681974,0.000117,0.000086,0.000044,0.000039,0.000043,0.000012,-0.000004,0.000008
EF v3.1 climate change: fossil global warming potential (GWP100),0.055665,0.143951,0.021438,0.071319,2.076421,0.039542,16762.513647,0.136820,0.097352,0.132666,0.208833,0.047676,-0.085988,0.003806,0.024093
EF v3.1 climate change: land use and land use change global warming potential (GWP100),0.085442,0.000035,0.000003,0.007521,0.007289,0.000009,41.465602,0.016757,0.014433,0.000176,0.089811,0.000165,0.087989,0.000046,0.015274


In [155]:
# Exporting to Excel
my_output.to_excel('Results_SA1e.xlsx')

In [33]:
my_output = pd.DataFrame(Results_SA1b, index=methods_names)
my_output.head()

,DME,NH3_CCS,bioMeOH,DAC,eH2,eMeOH_DOC,PO_slow,PO_fast,eMeOH_DAC,VLSFO,LNG,eMeOH_bio,eNH3
EF v3.1 acidification accumulated exceedance (AE),0.000063,0.000293,0.000039,0.000006,0.008782,-4868.553951,0.000654,0.000897,0.000450,0.000131,0.000030,0.000112,0.000215
EF v3.1 climate change global warming potential (GWP100),0.022528,0.152336,0.012078,0.003852,1.825471,16878.299988,0.014297,0.016549,0.127048,0.039637,0.021839,0.024737,0.042271
EF v3.1 climate change: biogenic global warming potential (GWP100),0.000007,-0.007563,0.000005,-0.000004,-0.000018,20.747884,0.000090,0.000013,0.000010,0.000008,0.000001,0.000003,0.000010
EF v3.1 climate change: fossil global warming potential (GWP100),0.022488,0.159865,0.009935,0.003810,1.819017,16815.954309,-0.083726,0.002336,0.126880,0.039619,0.021835,0.023564,0.042115
EF v3.1 climate change: land use and land use change global warming potential (GWP100),0.000033,0.000033,0.002138,0.000046,0.006472,41.597794,0.097933,0.014199,0.000158,0.000009,0.000003,0.001169,0.000147


In [34]:
# Exporting to Excel
my_output.to_excel('Results_SA1b.xlsx')

In [35]:
my_output = pd.DataFrame(Results_SA1c, index=methods_names)
my_output.head()

,LNG,eMeOH_DOC,eNH3,DAC,eH2,NH3_CCS,bioMeOH,DME,PO_fast,VLSFO,eMeOH_DAC,eMeOH_bio,PO_slow
EF v3.1 acidification accumulated exceedance (AE),0.000030,-4868.554011,0.000162,0.000006,0.006405,0.000316,0.000041,0.000069,0.000902,0.000132,0.000397,8.924648e-05,0.000666
EF v3.1 climate change global warming potential (GWP100),0.021839,16878.291186,0.034616,0.003852,1.480203,0.167082,0.013599,0.025955,0.019568,0.039768,0.119402,2.203010e-02,0.022041
EF v3.1 climate change: biogenic global warming potential (GWP100),0.000001,20.747878,0.000003,-0.000004,-0.000292,-0.007565,0.000004,0.000006,0.000013,0.000008,0.000004,5.559645e-07,0.000089
EF v3.1 climate change: fossil global warming potential (GWP100),0.021835,16815.945546,0.034493,0.003810,1.475254,0.174631,0.011458,0.025919,0.005360,0.039750,0.119268,2.087389e-02,-0.075972
EF v3.1 climate change: land use and land use change global warming potential (GWP100),0.000003,41.597763,0.000119,0.000046,0.005240,0.000016,0.002136,0.000029,0.014196,0.000009,0.000131,1.155648e-03,0.097924


In [36]:
# Exporting to Excel
my_output.to_excel('Results_SA1c.xlsx')

In [37]:
my_output = pd.DataFrame(Results_SA1d, index=methods_names)
my_output.head()

,PO_fast,eH2,eMeOH_DAC,DME,eNH3,eMeOH_DOC,VLSFO,LNG,eMeOH_bio,bioMeOH,PO_slow,DAC,NH3_CCS
EF v3.1 acidification accumulated exceedance (AE),0.000898,0.005614,0.000380,0.000064,0.000144,-4868.554031,0.000131,0.000030,8.017568e-05,0.000039,0.000656,0.000006,0.000296
EF v3.1 climate change global warming potential (GWP100),0.014331,1.365346,0.116859,0.020010,0.032070,16878.288259,0.039541,0.021839,1.950723e-02,0.010961,0.008608,0.003852,0.141502
EF v3.1 climate change: biogenic global warming potential (GWP100),0.000015,-0.000382,0.000002,0.000008,0.000001,20.747875,0.000009,0.000001,1.710839e-07,0.000005,0.000094,-0.000004,-0.007555
EF v3.1 climate change: fossil global warming potential (GWP100),0.000111,1.360898,0.116735,0.019962,0.031958,16815.942631,0.039523,0.021835,1.835328e-02,0.008815,-0.089433,0.003810,0.148998
EF v3.1 climate change: land use and land use change global warming potential (GWP100),0.014204,0.004830,0.000122,0.000039,0.000110,41.597752,0.000010,0.000003,1.153777e-03,0.002141,0.097946,0.000046,0.000058


In [38]:
# Exporting to Excel
my_output.to_excel('Results_SA1d.xlsx')

In [63]:
Results_SA2 = []
for m in EF31:
    results_SA2 = getLCAresults(Fuels_SA2,m) # total impact per tech
    Results_SA2.append(results_SA2)

In [64]:
my_output = pd.DataFrame(Results_SA2, index=methods_names)
my_output.head()

,bioMeOH,eH2,Wood,PO_fast,eMeOH_bio,PO_slow,Straw,DME
EF v3.1 acidification accumulated exceedance (AE),0.001615,0.010953,0.006274,0.001976,0.000987,0.006337,0.005379,0.001977
EF v3.1 climate change global warming potential (GWP100),0.383186,2.085421,1.926723,0.320433,0.226470,1.491988,1.578150,0.473274
EF v3.1 climate change: biogenic global warming potential (GWP100),0.000125,0.001711,0.000261,0.000047,0.000084,0.000234,0.000227,0.000164
EF v3.1 climate change: fossil global warming potential (GWP100),0.382891,2.076421,1.925823,0.320269,0.226220,1.491286,1.577458,0.472894
EF v3.1 climate change: land use and land use change global warming potential (GWP100),0.000171,0.007289,0.000640,0.000117,0.000166,0.000468,0.000465,0.000216


In [65]:
# Exporting to Excel
my_output.to_excel('Results_SA2B.xlsx')